In [2]:
import gymnasium as gym
import torch
import torch.nn as nn

In [3]:
env = gym.make('CartPole-v1', render_mode = 'rgb_array', max_episode_steps= 1000)

In [4]:
envs = gym.envs.registry
sorted(envs.keys())[:5] + ['...']

['Acrobot-v1', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', '...']

In [5]:
envs['CartPole-v1']

EnvSpec(id='CartPole-v1', entry_point='gymnasium.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, disable_env_checker=False, kwargs={}, namespace=None, name='CartPole', version=1, additional_wrappers=(), vector_entry_point='gymnasium.envs.classic_control.cartpole:CartPoleVectorEnv')

In [6]:
obs, info = env.reset(seed=0)

In [7]:
obs

array([ 0.01369617, -0.02302133, -0.04590265, -0.04834723], dtype=float32)

In [8]:
info

{}

In [9]:
img = env.render()
img.shape

DependencyNotInstalled: pygame is not installed, run `pip install "gymnasium[classic-control]"`